## Generate Codenames Clues using FastText

In [1]:
# ## Installs

# !pip install tqdm
# !pip install nltk

In [13]:
## Import 

import numpy as np
from numpy.linalg import norm
from gensim.models import KeyedVectors
import gensim.downloader
from random import shuffle
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

from tqdm.notebook import tqdm

In [3]:
# ## ** Run once to download FastText model

# fasttext = gensim.downloader.load('fasttext-wiki-news-subwords-300')
# fasttext.save('../data/fasttext_vectors.kv')

In [14]:
## Initialize 

# Initialize stemmers for word similarity
porter = PorterStemmer()
lancaster=LancasterStemmer()

# Load fastText vectors
fasttext = KeyedVectors.load('../data/fasttext_vectors.kv')

# print(len(fasttext))
# print(type(fasttext))
# print(len(fasttext[0]))

999999
<class 'gensim.models.keyedvectors.KeyedVectors'>
300


In [15]:
## Parameters

CODENAME_WORDS_FILE = '../data/codename_words.txt'
HINT_WORDS_FILE = '../data/hint_words.csv'
SUBSTR_LEN = 4

# # Modifiers for Clue Quality Metric (not used in this implementation)
# team_score_modifier = 10
# enemy_score_modifier = -5
# neutral_score_modifier = -1
# double_agent_score_modifier = -10

In [16]:
## Utility Functions

def cosine_similarity(x,y):
# Calculates cosine similarity of two word vectors
# via a normalized dot product 
    dp = np.dot(x,y) / (norm(x)*norm(y))
    return dp


def get_example_board(codenames_words):
# Returns a list of lists representing game board
# [[team_words], [enemy_words], [neutral_words], [double_agent_word]]
    words = codenames_words.copy()
    shuffle(words)
    return [
        words[0:9], #team words - 9
        words[9:17], #enemy words - 8
        words[17:24], #neutral words - 7
        [words[24]] #double agent word - 1
    ]


def load_hint_words():
# Returns a list of hint words read from csv file
# Note: Later change this to the list of words in fasttext
    hint_words = pd.read_csv(HINT_WORDS_FILE, index_col=0).hints.tolist()
    hint_words = remove_unseen_words(hint_words)
    return hint_words


def load_codename_words():
# Returns a list of words from codenames 
# Turns words lower case and removes spaces from compound words
# Removes a word from the game if it's not in fasttext model
    with open(CODENAME_WORDS_FILE, 'r') as f:
        uppercase_words = f.readlines()

    # Turn words lower case and remove spaces from compound words
    game_words = [w.lower().strip().replace(" ", "") for w in uppercase_words]
    
    # Removes a word from game if it has no corresponding fasttext word vector
    game_words = remove_unseen_words(game_words)
    
    # Return a list of game words
    return game_words


def remove_unseen_words(inlist):
# Removes words from list that are not found in fasttext model
# Returns a "cleaned" copy of list
# Note: original list passed in remains unaltered
    outlist = []
    for w in inlist:
        if w in fasttext.key_to_index:
            outlist.append(w)
        else:
            print("[remove_unseen_words] OUTPUT: %s not found in fasttext -> excluded from words list"%w)
            continue
    return outlist


def long_comm_substr(s1, s2):
# Returns the length of the longest common substring of two strings
    res = 0;
    for a in range(len(s1)):
        for b in range(len(s2)):
            k = 0;
            while (a+k) < len(s1) and (b+k)<len(s2) and s1[a+k]==s2[b+k]:
                k = k + 1;
            res = max(res, k);
    return res


def are_similar(w1, w2):
# Returns true if two words are "similar"
# Two words are similar if they have the same stem, share a common substr, or if one contains the other
# Porter stemming and Lancaster stemming are used via NLTK package
    if w1.__contains__(w2) or w2.__contains__(w1):
        print("[are_similar] OUTPUT: %s and %s are similar: one contains the other"%(w1,w2))
        return True
    elif porter.stem(w1) == porter.stem(w2):
        print("[are_similar] OUTPUT: %s and %s are similar: Porter Stemmer"%(w1,w2))
        return True
    elif lancaster.stem(w1) == lancaster.stem(w2):
        print("[are_similar] OUTPUT: %s and %s are similar: Lancaster Stemmer"%(w1,w2))
        return True
    elif long_comm_substr(w1,w2) >= SUBSTR_LEN:
        print("[are_similar] OUTPUT: %s and %s are similar: longest common substring too long"%(w1,w2))
        return True
    else:
        return False
    
    

In [17]:
## Main functions

def create_sim_mat(hint_words, game_words):
# Stores the similarities of hint/game word combinations in a .csv file

    # Create a pandas df to store similarities of each game word/hint word pair
    data = pd.DataFrame(np.zeros((len(hint_words), len(game_words))))

    # For each hint word
    for i in tqdm(range(len(hint_words))):

        # Find fasttext word vector for hint word
        hwv = fasttext[hint_words[i]]

        # For each game word
        for j in range(len(game_words)):

            # Find fasttext word vector for hint word
            gwv = fasttext[game_words[j]]

            # Calculate and store cos similarity of game word and hint word
            data.iloc[i, j] = cosine_similarity(gwv, hwv)

    print(data.head())
    
    # Save dataframe to .csv file
    data.index = hint_words
    data.columns = game_words
    data.index = data.index.str.lower()
    data.columns = data.columns.str.lower()
    data.to_csv('../data/similarity_matrix.csv')

In [8]:
# ## ** Run this cell only once ** 
# ## Create similarity matrix

# # Load the hint words and the game (codenames) words
# hint_words = load_hint_words()
# game_words = load_codename_words()

# # Store similarity scores of hint/game words in .csv file
# create_sim_mat(hint_words, game_words)


In [21]:
# 1. for every hint word, find the most similar game words.
# 2. score the hint word based on which game words are most similar 
    # (our color is most similar, enemy color/assassin are most dissimilar)
# 3. return the word based on the best score

def hint_giver(sim_mat, board):
    
    # Unpack board info into separate lists
    team_words, enemy_words, neutral_words, double_agent_word = board
    
    # Get a list of all game words
    board_words = team_words + enemy_words + neutral_words + double_agent_word
    
    # Keep track of "best" hint words
    best_score = 0
    best_words = []
    best_corr_words = []
    
    # Get a similarity matrix for hint words x board words
    game_similarity = sim_mat.loc[:, board_words]
    
    
    # For each row (hint)
    for row in game_similarity.iterrows():
        
        # Get hint word
        hint = row[0]
        
#         # If hint is too similar to any word on board, disregard it
#         for board_word in board_words:
#             if are_similar(hint, board_word):
#                 print("Not yet implemented")

        # Add thresholding to make better hints
        
        # Sort similarity scores for this hint (for each game word)
        df = row[1].sort_values(ascending=False)
        
        # Store score for this hint
        score = 0
        corr_words = []

        # For each game word, in descending order of highest similarity
        for game_word in df.index.tolist():
            # If word belongs to team, increase score of this hint
            if game_word in team_words:
                score += 1
                corr_words.append(game_word)
                continue
            # If word does not belong to team, stop increasing score for this hint
            break

        # Update best score
        if best_score < score: 
            best_score = score
            best_words = [hint]
            best_corr_words = [corr_words]
        elif best_score == score: # add if tied
            best_words.append(hint)
            best_corr_words.append(corr_words)
    
        # Implement some kind of tie breaker
        # ...
        
    return best_words, best_score, best_corr_words


# for integration: output as (hintword, hintnumber)

In [19]:
## Test Cell

# similarity_matrix = pd.read_csv('../data/similarity_matrix.csv', index_col=0).drop_duplicates()
# print(similarity_matrix.shape)

# hint_words = load_hint_words()
# game_words = load_codename_words()

# board = get_example_board(game_words)
# print(board)

# print("Giving hint...")
# hint = hint_giver(similarity_matrix, board)
# print(hint) 


(8781, 398)
[remove_unseen_words] OUTPUT: pdas not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: zus not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: anaheim not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: mpegs not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: greensboro not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: usps not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: mrna not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: cdna not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: liechtenstein not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: swaziland not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: lochness not found in fasttext -> excluded from words list
[remove_unseen_words] OUTP

In [20]:
# # Scratch Cell

# are_similar("happy", "happier")
# are_similar("clouds", "cloudy")
# are_similar("trouble", "troubling")
# are_similar("troubling", "trout")
# are_similar("nineteen", "eighteen")


[are_similar] OUTPUT: happy and happier are similar: Lancaster Stemmer
[are_similar] OUTPUT: clouds and cloudy are similar: longest common substring too long
[are_similar] OUTPUT: trouble and troubling are similar: Porter Stemmer
[are_similar] OUTPUT: troubling and trout are similar: longest common substring too long
[are_similar] OUTPUT: nineteen and eighteen are similar: longest common substring too long


True